In [1]:
import ast
import json
import random
import time

import autopep8
import numpy as np
import pandas as pd
from IPython.display import display
from pandas.api.types import is_numeric_dtype

# do get rid of annoying warnings
pd.set_option('future.no_silent_downcasting', True)

In [2]:
# global, for the print statements
verbose = True

# this should probably be on a seperate cell so I don't constantly reload the dataframes
fDefPath = 'reviews/yelpReviews/yelp_academic_dataset_'
# constants so I don't have to keep changing names
BS = 'business'
CH = 'checkin'
TI = 'tip'
RW = 'review'
US = 'user'

# subsets of what i care about
bssub = ['business_id', 'postal_code',
         'review_count', 'attributes', 'categories']
ussub = ['user_id', 'review_count', 'yelping_since']
rwsub = ['user_id', 'business_id', 'stars', 'text', 'date']

# constants for the file path
bspath = f'{fDefPath}{BS}.json'
chpath = f'{fDefPath}{CH}.json'
tipath = f'{fDefPath}{TI}.json'
rwpath = f'{fDefPath}{RW}.json'
uspath = f'{fDefPath}{US}.json'

In [3]:
# load the datasets
bsdf = pd.read_json(bspath, lines=True)
# take only what we want
bsdf = bsdf[bssub]

In [4]:
# create a manual dictionary for now
category_map = {
    # removed food, too broad, was catching grocery stores
    'restaurants': {'restaurants', 'sandwiches', 'american (new)', 'american (traditional)', 'pizza', 'bakeries', 'desserts', 'cafes', 'breakfast & brunch'},
    'retail': {'shopping', 'home & garden', 'fashion', 'real estate', 'grocery', 'department stores', 'electronics', 'toys'},
    'beauty': {'beauty & spas', 'hair salons', 'nail salons', 'massage', 'skin care', 'makeup artists'},
    'auto': {'automotive', 'auto repair'},
    'services': {'home services', 'local services', 'event planning & services'},
    'fitness': {'active life', 'fitness & instruction', 'gyms', 'yoga', 'martial arts'},
    'health': {'health & medical', 'doctors', 'dentists'},
    'pets': {'veterinarians', 'pet services', 'pets'},
    'nightlife': {'nightlife', 'bars', 'lounges', 'dance clubs', 'karaoke'},
    'travel': {'hotels & travel', 'hotels', 'tours'},
    'religious': {'religious organizations', 'synagogues'},
    'entertainment': {'festivals', 'arts & entertainment', 'television', 'mass media', 'museums', 'music venues', 'cinema', 'theaters'},
    'finance': {'banks & credit unions', 'financial services'},
    'legal': {'dui law', 'lawyers'},
    'government': {'public services & government', 'libraries'},
    'education': {'education'},
    'professional services': {'employment agencies', 'professional services'}
}

In [6]:
def matchBroadCategory(cat_list):
    for broad_cat, keywords in category_map.items():
        if any(cat in keywords for cat in cat_list):
            return broad_cat
    return "other"


# expand categories so we can then sort them into broader categories
bsdf = bsdf.dropna(subset=['categories', 'attributes']).copy()
bsdf['categories'] = bsdf['categories'].str.lower()
bsdf.loc[:, "categoryList"] = bsdf.loc[:, "categories"].str.split(", ")

# Flatten and count unique categories
allCategories = bsdf["categoryList"].explode().str.lower()
categoryCounts = allCategories.value_counts()

# Preview common categories
# print(categoryCounts.head(50))

bsdf["broadCategory"] = bsdf["categoryList"].apply(matchBroadCategory)

In [7]:
# put this into a csv so i can look at them better, only if verbose is True
if verbose:
    fileName = 'files/categorized.csv'
    bsdf[['categoryList', 'broadCategory']].to_csv(fileName)

In [8]:
# now I'm going to process the attributes in the business json
attDf = bsdf.dropna(subset=['attributes']).copy()

In [9]:
baseDf = bsdf[['business_id', 'attributes']].copy()
attDf = baseDf.attributes
# print(attDf.head())
attDf = pd.json_normalize(attDf, sep='_')

# normalize converts nested dictionaries into strings,
# convert it back into a dictionary
for col in attDf.columns:
    if attDf[col].apply(lambda x: isinstance(x, str) and x.strip().startswith('{')).any():
        attDf[col] = attDf[col].apply(
            lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Find columns with nested dicts
nested_cols = [col for col in attDf.columns if isinstance(
    attDf.dropna(subset=[col])[col].iloc[0], dict)]

# Flatten each nested dict into new columns
for col in nested_cols:
    nested_df = pd.json_normalize(attDf[col])
    nested_df.columns = [f"{col}_{subcol}" for subcol in nested_df.columns]
    attDf = attDf.drop(columns=[col]).join(nested_df)

In [10]:
# merge attributes back with their business_id
# drop old attribute column
if len(baseDf.columns) > 1:
    baseDf = baseDf.drop(['attributes'], axis=1)

# remove index to make sure they line up
baseDf = baseDf.reset_index(drop=True)
attDf = attDf.reset_index(drop=True)
# merge
mergedDf = pd.concat([baseDf, attDf], axis=1)
if verbose:
    print(mergedDf.head())

              business_id ByAppointmentOnly BusinessAcceptsCreditCards  \
0  Pns2l4eNsfO8kk83dixA6A              True                        NaN   
1  mpf3x-BjTdTEA3yCZrAYPw               NaN                       True   
2  tUFrWirKiKi_TAnsVWINQQ             False                       True   
3  MTSW4McQd7CbVtyjqoe9mw             False                      False   
4  mWMc6_wTdE0EUBKIGXDVfA               NaN                       True   

  BikeParking RestaurantsPriceRange2 CoatCheck RestaurantsTakeOut  \
0         NaN                    NaN       NaN                NaN   
1         NaN                    NaN       NaN                NaN   
2        True                      2     False              False   
3        True                      1       NaN               True   
4        True                    NaN       NaN               True   

  RestaurantsDelivery Caters     WiFi  ... HairSpecializesIn_kids  \
0                 NaN    NaN      NaN  ...                    NaN   
1 

In [11]:
# write at attributes normalized dataframe and the merged dataframe for
# easier manual checking (they're so long they don't fit in print statements)
if verbose:
    attDf.to_csv('files/att.csv')
    mergedDf.to_csv('files/mergedAtt.csv')

In [12]:
# doesn't manipulate data, just for checking. Will do nothing if verbose == False
if verbose:
    # check to make sure the business_id was lining up
    indices = list(range(len(baseDf)))
    random.shuffle(indices)
    # compare 15 random rows, using business_is as a common index
    lookat = indices[:15]

    # probably a cleaner way to do this, but oh well
    for ind in lookat:
        bsid = baseDf.iloc[ind]['business_id']
        mgid = mergedDf.iloc[ind]['business_id']
        bsAtt = list(bsdf.loc[bsdf['business_id'] == bsid]['attributes'])
        mdAtt = mergedDf.loc[mergedDf['business_id']
                             == mgid].drop(['business_id'], axis=1)
        mdAtt = mdAtt.dropna(axis=1)
        print(f' {bsid} : {mgid}')
        print(f'base frame att :\n {bsAtt} \n merged frame att: \n {mdAtt} ')

 SQt8mGJsvGQrIIjEi7eOrg : SQt8mGJsvGQrIIjEi7eOrg
base frame att :
 [{'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'BusinessAcceptsCreditCards': 'True', 'ByAppointmentOnly': 'False', 'RestaurantsPriceRange2': '2', 'AcceptsInsurance': 'False', 'WiFi': "u'no'", 'BikeParking': 'True', 'RestaurantsDelivery': 'None', 'RestaurantsTakeOut': 'None'}] 
 merged frame att: 
        ByAppointmentOnly BusinessAcceptsCreditCards BikeParking  \
125172             False                       True        True   

       RestaurantsPriceRange2 RestaurantsTakeOut RestaurantsDelivery   WiFi  \
125172                      2               None                None  u'no'   

       AcceptsInsurance BusinessParking_garage BusinessParking_street  \
125172            False                  False                  False   

       BusinessParking_validated BusinessParking_lot BusinessParking_valet  
125172                     False                True   

In [13]:
# see which columns are binary and which are categorical
# replace 'None' and None with np.nan
# replace true and false with binary
mergedDf.replace(['None', 'none', 'NONE', None], np.nan, inplace=True)
mergedDf.replace(['True', True, 'False', False], [1, 1, 0, 0], inplace=True)
# convert dtypes so is_numeric_dtype works (they're all object dtype right now)_
mergedDf = mergedDf.convert_dtypes()

for col in mergedDf.columns:
    if col == 'business_id':
        continue

    if verbose:
        print(
            f'{col} has attributes is of type {mergedDf[col].dtype}: {mergedDf[col].unique()} : trimming . . .')

    if is_numeric_dtype(mergedDf[col].dtype):
        if verbose:
            print('is numeric')
    else:
        mergedDf[col] = mergedDf[col].str.strip('u\'')

    if verbose:
        print(f'{col} has attributes: {mergedDf[col].unique()}\n\n')

ByAppointmentOnly has attributes is of type Int64: <IntegerArray>
[1, <NA>, 0]
Length: 3, dtype: Int64 : trimming . . .
is numeric
ByAppointmentOnly has attributes: <IntegerArray>
[1, <NA>, 0]
Length: 3, dtype: Int64


BusinessAcceptsCreditCards has attributes is of type Int64: <IntegerArray>
[<NA>, 1, 0]
Length: 3, dtype: Int64 : trimming . . .
is numeric
BusinessAcceptsCreditCards has attributes: <IntegerArray>
[<NA>, 1, 0]
Length: 3, dtype: Int64


BikeParking has attributes is of type Int64: <IntegerArray>
[<NA>, 1, 0]
Length: 3, dtype: Int64 : trimming . . .
is numeric
BikeParking has attributes: <IntegerArray>
[<NA>, 1, 0]
Length: 3, dtype: Int64


RestaurantsPriceRange2 has attributes is of type string: <StringArray>
[<NA>, '2', '1', '3', '4']
Length: 5, dtype: string : trimming . . .
RestaurantsPriceRange2 has attributes: <StringArray>
[<NA>, '2', '1', '3', '4']
Length: 5, dtype: string


CoatCheck has attributes is of type Int64: <IntegerArray>
[<NA>, 0, 1]
Length: 3, dtype: I

In [14]:
# merged mergedDf (which is just business_id and  attributes(normalized) right now)
# with the broad categories
# drop unecessary columns (if they haven't been already)
if 'attributes' in bsdf.columns:
    bsdf = bsdf.drop(['attributes', 'categories', 'categoryList'], axis=1)

busCleanedDf = mergedDf.merge(bsdf, on='business_id', how='left')

if verbose:
    pd.set_option('display.max_columns', None)
    display(busCleanedDf.head(3))
    pd.reset_option('display.max_columns')

# clean it up a bit, remove columns that are mostly empty
threshold = 0.9
busCleanedDf = busCleanedDf.loc[:, busCleanedDf.isnull().mean() < threshold]
busCleanedDf.to_csv('files/busCleaned.csv')

,business_id,ByAppointmentOnly,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,CoatCheck,RestaurantsTakeOut,RestaurantsDelivery,Caters,WiFi,WheelchairAccessible,HappyHour,OutdoorSeating,HasTV,RestaurantsReservations,DogsAllowed,Alcohol,GoodForKids,RestaurantsAttire,RestaurantsTableService,RestaurantsGoodForGroups,DriveThru,NoiseLevel,BusinessAcceptsBitcoin,Smoking,GoodForDancing,AcceptsInsurance,BYOB,Corkage,BYOBCorkage,Open24Hours,RestaurantsCounterService,AgesAllowed,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,Ambience_romantic,Ambience_intimate,Ambience_touristy,Ambience_hipster,Ambience_divey,Ambience_classy,Ambience_trendy,Ambience_upscale,Ambience_casual,GoodForMeal_dessert,GoodForMeal_latenight,GoodForMeal_lunch,GoodForMeal_dinner,GoodForMeal_brunch,GoodForMeal_breakfast,Music_dj,Music_background_music,Music_no_music,Music_jukebox,Music_live,Music_video,Music_karaoke,BestNights_monday,BestNights_tuesday,BestNights_friday,BestNights_wednesday,BestNights_thursday,BestNights_sunday,BestNights_saturday,HairSpecializesIn_straightperms,HairSpecializesIn_coloring,HairSpecializesIn_extensions,HairSpecializesIn_africanamerican,HairSpecializesIn_curly,HairSpecializesIn_kids,HairSpecializesIn_perms,HairSpecializesIn_asian,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,postal_code,review_count,broadCategory
0,Pns2l4eNsfO8kk83dixA6A,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,93101,7,health
1,mpf3x-BjTdTEA3yCZrAYPw,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,63123,15,services
2,tUFrWirKiKi_TAnsVWINQQ,0,1,1,2,0,0,0,0,no,1,0,0,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,0,1,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,85711,22,retail
